reference.

https://python.langchain.com/docs/expression_language/get_started

In [ ]:
%pip install faiss-cpu

In [15]:
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings

import os
from dotenv import load_dotenv


load_dotenv(override=True)

# check if USE_OLLAMA is True
if os.getenv('USE_OLLAMA') == 'True':
    print('Using OLLAMA')
if os.getenv('USE_OPENAI') == 'True':
    print('Using OpenAI') 


Using OLLAMA


Facebook AI Similarity Search (Faiss) is a library for efficient similarity search and clustering of dense vectors.

search result will be fetched by retriever and it'll be used as context

In [16]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import GPT4AllEmbeddings


raw_documents = TextLoader('./private_text.txt').load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(raw_documents)

vectorstore = FAISS.from_documents(documents, embedding=GPT4AllEmbeddings())

retriever = vectorstore.as_retriever()

make a template with context

In [17]:
from langchain_community.llms import Ollama
from langchain_community.chat_models import ChatOllama

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
#llm = Ollama(model="llama2")
llm = ChatOllama(model="llama2")

#model = ChatOpenAI(openai_api_key=os.environ["OPENAI_API_KEY"], model_name="gpt-4-0125-preview")

output_parser = StrOutputParser()


context include custom test resutls fetched by the retriever. 
question will contain user's question.

In [18]:

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt 
    | llm 
    | output_parser
)

chain.invoke("tell me about the funnyIO")

'Based on the provided context, FunnyIO appears to be a high-performance and scalable host controller interface designed for accessing solid-state drives (SSDs) over a PCIExpress (PCIe) bus. Developing FunnyIO drivers is crucial for enabling efficient communication between the operating system and FunnyIO storage devices.\n\nSome key aspects of FunnyIO driver development include:\n\n1. Understanding the FunnyIO protocol: The FunnyIO protocol is designed to take advantage of the low latency and parallelism of modern SSDs. It provides efficient command submission and completion mechanisms, enabling high-speed data transfers between the host and the storage device.\n2. PCIe interface: FunnyIO drivers interact with FunnyIO storage devices over a PCIe interface. Therefore, understanding PCIe bus architecture and communication protocols is essential for developing efficient FunnyIO drivers.\n3. Memory management: FunnyIO drivers need to manage memory resources efficiently to optimize data tr